In [0]:
from pyspark.sql.types import *
import  pyspark.sql.functions as F
import time     

In [0]:
conf = {}
connectionString = "Endpoint=sb://azureproject-ehs.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=PkhHwefVkc1YRM9RBmdft7FBSUKud1Jkf5z/jisV4QQ=;EntityPath=iot_events_raw"
conf["eventhubs.connectionString"] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)
# change connection string to yours from Event Hub

read_df = (
  spark
    .readStream
    .format("eventhubs")
    .options(**conf)
    .load()
)



In [0]:
schema = StructType([StructField("body", StringType(), True)])
coinEventParsedDf = (read_df.select(F.from_json(F.col("body")
                                                      .cast("string"),schema)
                                                      .alias("message"))
                                                      .withColumn("receivedTime",F.current_timestamp()))


In [0]:
(coinEventParsedDf
              .writeStream
              .format("delta")                        
              .outputMode("append")
              .option("checkpointLocation","/ucu_cluster2/coinPriceCheck/checkpoints/json-events")
              .queryName("coinPrice")
              .start("/ucu_cluster2/coinPriceCheck"))

Out[26]: <pyspark.sql.streaming.StreamingQuery at 0x7f6b1e5ce190>

In [0]:
#create a Delta table
spark.sql("CREATE TABLE coinPriceCheck USING DELTA LOCATION '/ucu_cluster2/coinPriceCheck'")

Out[29]: DataFrame[]

In [0]:
%sql select * from coinPriceCheck

message receivedTime List({"status":{"timestamp":"2021-08-05T17:46:00.706Z","error_code":0,"error_message":null,"elapsed":561,"credit_count":1,"notice":"You have used 320% of your plan's daily credit limit."},"data":{"ADA":{"id":2010,"name":"Cardano","symbol":"ADA","slug":"cardano","num_market_pairs":284,"date_added":"2017-10-01T00:00:00Z","tags":["mineable","dpos","pos","platform","research","smart-contracts","staking","binance-smart-chain"],"max_supply":45000000000,"circulating_supply":3.2081180259179E10,"total_supply":3.2959337588354E10,"is_active":1,"platform":null,"cmc_rank":5,"is_fiat":0,"last_updated":"2021-08-05T17:45:08Z","quote":{"USD":{"price":1.3869059331113,"volume_24h":1.765915448956661E9,"percent_change_1h":0.78489221,"percent_change_24h":0.28377568,"percent_change_7d":8.15307874,"percent_change_30d":-1.40009141,"percent_change_60d":-17.47626133,"percent_change_90d":-17.42372458,"market_cap":4.449357924266847E10,"last_updated":"2021-08-05T17:45:08Z"}}},"BTC":{"id":1,"name":"Bitcoin","symbol":"BTC","slug":"bitcoin","num_market_pairs":8942,"date_added":"2013-04-28T00:00:00Z","tags":["mineable","pow","sha-256","store-of-value","state-channels","coinbase-ventures-portfolio","three-arrows-capital-portfolio","polychain-capital-portfolio","binance-labs-portfolio","arrington-xrp-capital","blockchain-capital-portfolio","boostvc-portfolio","cms-holdings-portfolio","dcg-portfolio","dragonfly-capital-portfolio","electric-capital-portfolio","fabric-ventures-portfolio","framework-ventures","galaxy-digital-portfolio","huobi-capital","alameda-research-portfolio","a16z-portfolio","1confirmation-portfolio","winklevoss-capital","usv-portfolio","placeholder-ventures-portfolio","pantera-capital-portfolio","multicoin-capital-portfolio","paradigm-xzy-screener"],"max_supply":21000000,"circulating_supply":18777093,"total_supply":18777093,"is_active":1,"platform":null,"cmc_rank":1,"is_fiat":0,"last_updated":"2021-08-05T17:45:02Z","quote":{"USD":{"price":40342.51842024975,"volume_24h":3.1725229383243385E10,"percent_change_1h":2.63391717,"percent_change_24h":2.4861728,"percent_change_7d":1.54490411,"percent_change_30d":19.07170958,"percent_change_60d":11.8307406,"percent_change_90d":-30.56961961,"market_cap":7.575152202312426E11,"last_updated":"2021-08-05T17:45:02Z"}}},"DOT":{"id":6636,"name":"Polkadot","symbol":"DOT","slug":"polkadot-new","num_market_pairs":229,"date_added":"2020-08-19T00:00:00Z","tags":["substrate","polkadot","binance-chain","polkadot-ecosystem","three-arrows-capital-portfolio","polychain-capital-portfolio","blockchain-capital-portfolio","boostvc-portfolio","cms-holdings-portfolio","coinfund-portfolio","fabric-ventures-portfolio","fenbushi-capital-portfolio","hashkey-capital-portfolio","kinetic-capital","1confirmation-portfolio","placeholder-ventures-portfolio","pantera-capital-portfolio","exnetwork-capital-portfolio"],"max_supply":null,"circulating_supply":9.810403473355887E8,"total_supply":1.0967645037607765E9,"is_active":1,"platform":null,"cmc_rank":9,"is_fiat":0,"last_updated":"2021-08-05T17:45:05Z","quote":{"USD":{"price":19.24354349210976,"volume_24h":1.723680489935389E9,"percent_change_1h":1.60741619,"percent_change_24h":-1.0243888,"percent_change_7d":28.28563265,"percent_change_30d":22.89229937,"percent_change_60d":-20.46324321,"percent_change_90d":-54.1237691,"market_cap":1.887869259146687E10,"last_updated":"2021-08-05T17:45:05Z"}}},"ETH":{"id":1027,"name":"Ethereum","symbol":"ETH","slug":"ethereum","num_market_pairs":5644,"date_added":"2015-08-07T00:00:00Z","tags":["mineable","pow","smart-contracts","ethereum","coinbase-ventures-portfolio","three-arrows-capital-portfolio","polychain-capital-portfolio","binance-labs-portfolio","arrington-xrp-capital","blockchain-capital-portfolio","boostvc-portfolio","cms-holdings-portfolio","dcg-portfolio","dragonfly-capital-portfolio","electric-capital-portfolio","fabric-ventures-portfolio","framework-ventures","hashkey-capital-portfolio","kinetic-capital","huobi-capital","ala

In [0]:
# create connection to storage account
storage_account_name = "azureprojectstorage"
storage_account_key = "ATwZgYpVG0hGRgzzvucBmz6d7KBSfQoXhnlI44frhVxk8oaPwIuR66j/ddVl2d1Nvu+K2Sj9PnUk0MwXLSdDDA=="
container = "contents"
spark.conf.set("fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name), storage_account_key)

output_container_path = "wasbs://%s@%s.blob.core.windows.net" % (container, storage_account_name)
output_blob_folder = "%s/coindata_data_folder" % output_container_path

df1 = spark.read.format("delta").load("/ucu_cluster2/coinPriceCheck")
df2 = df1.select(df1.message.cast('string').alias('message1'),df1.receivedTime)


(df2
 .coalesce(1)
 .write
 .mode("overwrite")
 .option("header", "true")
 .format("com.databricks.spark.csv")
 .save(output_blob_folder))

In [0]:
URI = "cosmos-db-uri"
PrimaryKey = "your-cosmos-db-key"CosmosDatabase = "AdventureWorks"
CosmosCollection = "ratings"writeConfig = {
  "Endpoint": URI,
  "Masterkey": PrimaryKey,
  "Database": CosmosDatabase,
  "Collection": CosmosCollection,
  "writingBatchSize":"1000",
  "Upsert": "true"
}

In [0]:
for s in spark.streams.active:
    s.stop()